# How-To


*   Run each cell after another, starting from the top
*   Hand in your CMEMS credentials 
*   Customize Route and algorithm parameters along the respective widget cell


# Initialization
- Installing and loading packages
- Clone GitHub repository
- Hand in CMEMS credentials
- Make route settings
  - Route start and end point (Geographic coordinates)
  - Start and end time of route
  - Genetic algorithm settings (generations, population, offspring)
  - Area of Interest (Geographi coordinates Boundingbox)
  - Draft of the ship

In [23]:
%cd "C:\Users\pgile\OneDrive\Documents\Thesis_Digital_twin\BOG_Digital_Twin\cms_routing"

import os
import sys

# This shows where the notebook thinks it is located
print(f"Current Directory: {os.getcwd()}")

C:\Users\pgile\OneDrive\Documents\Thesis_Digital_twin\BOG_Digital_Twin\cms_routing
Current Directory: C:\Users\pgile\OneDrive\Documents\Thesis_Digital_twin\BOG_Digital_Twin\cms_routing


In [1]:
from advanced_scripts import prediction

In [2]:
#@title
import ipywidgets as widgets
from ipywidgets import HBox, VBox

lon_st = widgets.FloatText(value=-73, description='lon_s:', step = 0.1)
lat_st = widgets.FloatText(value=40.7,description='lat_s:', step = 0.1)
lon_de = widgets.FloatText(value=-10,description='lon_d:', step = 0.1)
lat_de = widgets.FloatText(value=38.7,description='lat_d:', step = 0.1)
stTime = widgets.Text(value="01.12.2025 12:00", description='startTime:')
eTime = widgets.Text(value="10.12.2025 12:00", description='endTime:')
generation_count = widgets.IntText(value=100, description='generations:')
pop_size = widgets.IntText(value=20, description='population:')
offspring = widgets.IntText(value=10, description='offspring:')
lon_min = widgets.IntText(value=-80, description='lon_min:')
lon_max = widgets.IntText(value=0, description='lon_max')
lat_min = widgets.IntText(value=10, description='lat_min')
lat_max = widgets.IntText(value=60, description='lat_max')
draft = widgets.FloatSlider(value=12, steps=0.1, min=6.6, max=16, description='draft')

In [3]:
#@title
list_widgets  = [
    widgets.VBox([lon_st, lat_st, lon_de, lat_de]),
    widgets.VBox([stTime, eTime]),
    widgets.VBox([pop_size, offspring, generation_count]),
    widgets.VBox([lon_min, lon_max, lat_min, lat_max]),
    widgets.VBox([draft])
    ]
                
accordion = widgets.Accordion(children=list_widgets)
accordion.set_title(0, 'Route Definition:')
accordion.set_title(1, 'Start and End Time:')
accordion.set_title(2, 'Algorithm Settings:')
accordion.set_title(3, 'Area of Interest:')
accordion.set_title(4, 'Ship Parameter:')
accordion

Accordion(children=(VBox(children=(FloatText(value=-73.0, description='lon_s:', step=0.1), FloatText(value=40.…

In [4]:
# Assign variables from widget
lon_s = lon_st.value
lat_s = lat_st.value
lon_d = lon_de.value
lat_d = lat_de.value

AOI = [lon_min.value, lon_max.value, lat_min.value, lat_max.value] 

startTime=stTime.value
endTime=eTime.value

# Prepare data
- Download CMEMS data (takes some time)
- Prepare grids to calculate the routes on 

In [30]:
import importlib
import advanced_scripts.prediction as prediction
importlib.reload(prediction)

(ds_w, ds_p) = prediction.get_cmems(startTime, endTime, UN_CMEMS="pgiles",  PW_CMEMS= "@rCti5m050603")

print("Data loaded successfully!")

WARNING - 2026-01-05T01:28:01Z - 'force_download' has been deprecated.


Fetching data for midpoint date: 2025-12-06
Date is recent. Using Analysis & Forecast (NRT)...


INFO - 2026-01-05T01:31:55Z - Selected dataset version: "202411"
INFO - 2026-01-05T01:31:55Z - Selected dataset part: "default"
INFO - 2026-01-05T01:32:01Z - Starting download. Please wait...


  0%|          | 0/111 [00:00<?, ?it/s]

INFO - 2026-01-05T01:33:59Z - Successfully downloaded to cmems_data\wave_data_(3).nc
WARNING - 2026-01-05T01:33:59Z - 'force_download' has been deprecated.


INFO - 2026-01-05T01:39:18Z - Selected dataset version: "202406"
INFO - 2026-01-05T01:39:18Z - Selected dataset part: "default"
INFO - 2026-01-05T01:39:29Z - Starting download. Please wait...


  0%|          | 0/25 [00:00<?, ?it/s]

INFO - 2026-01-05T01:47:23Z - Successfully downloaded to cmems_data\physics_data.nc


Data loaded successfully!


In [5]:
import copernicusmarine

# Replace with the dataset_ID you are using (e.g., "cmems_mod_glo_phy_anfc_0.083deg_P1D-m")
info = copernicusmarine.describe(dataset_id="cmems_mod_glo_phy_anfc_0.083deg_P1D-m")
print(info)

Fetching catalogue 1:  50%|█████     | 1/2 [00:05<00:05,  5.95s/it]

products=[CopernicusMarineProduct(title='Global Ocean Physics Analysis and Forecast', product_id='GLOBAL_ANALYSISFORECAST_PHY_001_024', thumbnail_url='https://mdl-metadata.s3.waw3-1.cloudferro.com/metadata/thumbnails/GLOBAL_ANALYSISFORECAST_PHY_001_024.jpg', description='The Operational Mercator global ocean analysis and forecast system at 1/12 degree is providing 10 days of 3D global ocean forecasts updated daily. The time series is aggregated in time in order to reach a two full year’s time series sliding window.\n\nThis product includes daily and monthly mean files of temperature, salinity, currents, sea level, mixed layer depth and ice parameters from the top to the bottom over the global ocean. It also includes hourly mean surface fields for sea level height, temperature and currents. The global ocean output files are displayed with a 1/12 degree horizontal resolution with regular longitude/latitude equirectangular projection.\n\n50 vertical levels are ranging from 0 to 5500 meter

In [6]:
from joblib import load
model = load("models/DTR_model.joblib")
SOG_N = prediction.prepare_grid(ds_w, ds_p, draft.value, "N", model)
SOG_E = prediction.prepare_grid(ds_w, ds_p, draft.value, "E", model)
SOG_S = prediction.prepare_grid(ds_w, ds_p, draft.value, "S", model)
SOG_W = prediction.prepare_grid(ds_w, ds_p, draft.value, "W", model)

Fetching catalogue 1: 100%|██████████| 2/2 [00:09<00:00,  4.70s/it]


ValueError: node array from the pickle has an incompatible dtype:
- expected: {'names': ['left_child', 'right_child', 'feature', 'threshold', 'impurity', 'n_node_samples', 'weighted_n_node_samples', 'missing_go_to_left'], 'formats': ['<i8', '<i8', '<i8', '<f8', '<f8', '<i8', '<f8', 'u1'], 'offsets': [0, 8, 16, 24, 32, 40, 48, 56], 'itemsize': 64}
- got     : [('left_child', '<i8'), ('right_child', '<i8'), ('feature', '<i8'), ('threshold', '<f8'), ('impurity', '<f8'), ('n_node_samples', '<i8'), ('weighted_n_node_samples', '<f8')]

In [ ]:
# Derive grid coordinates from geographic coordinates
import xarray as xr
import numpy as np
def get_closest(array, value):
    return np.abs(array - value).argmin()

ds = xr.open_dataset(xr.backends.NetCDF4DataStore(ds_w))

start_lon = get_closest(ds.longitude.data, lon_s)
start_lat = ds.sizes["latitude"]-get_closest(ds.latitude.data, lat_s)
dest_lon = get_closest(ds.longitude.data, lon_d)
dest_lat = ds.sizes["latitude"]-get_closest(ds.latitude.data, lat_d)

AOI = [get_closest(ds.longitude.data, AOI[0]),get_closest(ds.longitude.data, AOI[1]), 2041 - get_closest(ds.latitude.data, AOI[3]), 2041 - get_closest(ds.latitude.data, AOI[2])]

startpoint = (start_lat-AOI[2], start_lon-AOI[0])
endpoint = (dest_lat-AOI[2], dest_lon-AOI[0])

In [ ]:
# Prepare grids for AOI, including different engine powers and different sizes depending on latitude
timeGrids= calculateTimeGrid(SOG_E, SOG_N, SOG_S, SOG_W, AOI)

# Run Algorithm
- Run NSGA II algorithm 
- Measure time needed to run the algorithm

In [ ]:
from runAlgorithm import runAlgorithm
from time import process_time

t_start = process_time() # time before the algorithm starts
# run actual algorithm
res = runAlgorithm(startpoint, endpoint, startTime, endTime, timeGrids, population=pop_size.value, offspring=offspring.value, generations=generation_count.value)
t_stop = process_time() # timepoint after the algorithm ended
time_lr = t_stop-t_start # time algorithm took
print("Elapsed time in seconds: ", time_lr)

In [ ]:
# save results and routes (change label if wanted)
np.save("./routes_advanced_20g_10d_40pop",res.X)
np.save("./values_advanced_20g_10d_40pop",res.F)

In [ ]:
results = res.F
routes = res.X

# Visualization
- Show routes on map
  - Best in-time route
  - Most energy efficient route (least fuel use)
- Plot pareto front
- Plot pareto fronts over generations

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
import matplotlib.colors
from matplotlib.lines import Line2D
from matplotlib import cm
route_minTime = routes[np.argmin(results[:,0], axis=0)][1]
route_minfuelUSe = routes[np.argmin(results[:,1], axis=0)][1]

route_minTime_Display =   np.stack(route_minTime, axis=-1)
route_minfuelUSe_Display =   np.stack(route_minfuelUSe, axis=-1)

timeGridsDisplay= np.where(timeGrids[1][2] > 999, np.nan, timeGrids[1][2])

def changeSpeedforVisualization(speed):
  # speed = np.where(speed==0.7, 0.5, speed)
  speed = np.where(speed==0.7, 0.5, speed)
  speed = np.where(speed==0.6, 0.2, speed)
  return speed


fig, ax = plt.subplots(figsize=(14,7))
im = ax.imshow(timeGridsDisplay, aspect='auto', vmin=np.min(timeGrids))#, vmax=40);
sc2 = ax.scatter(route_minfuelUSe_Display[1],route_minfuelUSe_Display[0], c=cm.YlGn(changeSpeedforVisualization(route_minfuelUSe_Display[2])), edgecolor='none')
sc1 = ax.scatter(route_minTime_Display[1],route_minTime_Display[0], c=cm.YlOrRd(changeSpeedforVisualization(route_minTime_Display[2])), edgecolor='none', label='fastest')

plt.legend()
ax.plot(startpoint[1], startpoint[0], 'k^', markersize=15)
ax.plot(endpoint[1], endpoint[0], 'k*', markersize=15)
fig.colorbar(im, ax=ax, orientation='horizontal', label='time per grid cell', shrink=0.8);

custom_lines = [Line2D([0], [0], color=cm.YlOrRd(0.2), lw=4),
                Line2D([0], [0], color=cm.YlOrRd(0.5), lw=4),
                Line2D([0], [0], color=cm.YlOrRd(0.8), lw=4),
                Line2D([0], [0], color=cm.YlGn(0.2), lw=4),
                Line2D([0], [0], color=cm.YlGn(0.5), lw=4),
                Line2D([0], [0], color=cm.YlGn(0.8), lw=4),]
ax.legend(custom_lines, ['in time (0.6)', 'in time (0.7)', 'in time (0.8)', 'fuel (0.6)','fuel (0.7)', 'fuel (0.8)'])

In [ ]:
import math
from datetime import time, timedelta
# Pareto Front
def transformTick(timeDifference):
    td = timedelta(minutes=timeDifference)
    return ".".join(str(td).split(':')[:2])
    

timeDiffs= results[:,0]
timeDiff = np.array([math.sqrt(x) for x in timeDiffs])

f1, ax1 = plt.subplots(1, figsize=(15,10))
plt.scatter(timeDiff,results[:,1])
a= ax1.get_xticks().tolist()
print(a)
result= map(transformTick, a)
ax1.set_xticklabels(result)
ax1.set_title("Objective Space")
ax1.set_xlabel('Time Difference')
ax1.set_ylabel('Fuel Use')
#plt.savefig(default_directory+"/results2016N6/pareto_front.png")
plt.show()

In [ ]:
# Create an empty list to save objective values per generation
# Needed for history 
f = []
# iterate over the generations
for generation in res.history:
 # retrieve the optimal for all objectives from the generation
 opt = generation.opt
 this_f = opt.get("F")
 f.append(this_f)

fNumpy = np.asarray(f)
history = fNumpy

In [ ]:
# add here the generations you want to see in the plot
generations2plot = [math.floor(generation_count.value*0.2), math.floor(generation_count.value*0.4), math.floor(generation_count.value*0.6), math.floor(generation_count.value*0.8),generation_count.value]
# make the plot
fig4, ax4 = plt.subplots(1)
# i - 1, because generation 1 has index 0
for i in generations2plot:
 plt.scatter(history[i-1][:,0],history[i-1][:,1])

plt.legend(generations2plot)
plt.show()